# Download and unzip the cats vs. dogs data set

In [ ]:
%%bash
wget -q https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
unzip -q cats_and_dogs_filtered.zip

# Start with the usual imports

In [ ]:
import time
import os
import copy
import torch as pt
import numpy as np
import matplotlib.pyplot as plt

device = pt.device("cuda:0" if pt.cuda.is_available() else "cpu")

# The `torchvision.transforms` include image data augmentation features.

In [ ]:
from torchvision import transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),                                 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}
data_transforms

The `datasets.ImageFolder` simplifies import of image data (as files) into a `DataLoader`

In [ ]:
from torchvision import datasets

data_dir = 'cats_and_dogs_filtered'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: pt.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=os.cpu_count())
              for x in ['train', 'validation']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes

class_names

# The methods in `torchvision.utils` implement common patterns in image data processing.

In [ ]:
from torchvision import utils

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = utils.make_grid(inputs)

def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

imshow(out, title=[class_names[x] for x in classes])

## Reusable, retrainable `MobileNet v2` architecture.

The MobileNet v2 architecture is based on an inverted residual structure where the input and output of the residual block are thin bottleneck layers opposite to traditional residual models which use expanded representations in the input. MobileNet v2 uses lightweight depthwise convolutions to filter features in the intermediate expansion layer. Additionally, non-linearities in the narrow layers were removed in order to maintain representational power.


![](https://pytorch.org/assets/images/mobilenet_v2_1.png)

![](https://pytorch.org/assets/images/mobilenet_v2_2.png)


In [ ]:
from torchvision.models import mobilenet_v2
model = mobilenet_v2(pretrained=True)
model

# Capture the number of the input features in the last `Linear` layer of the model.

In [ ]:
model.classifier[1]

In [ ]:
num_ftrs = model.classifier[1].in_features
num_ftrs

# We need to set `requires_grad == False` to freeze the parameters so that the gradients are not computed in `backward()`


In [ ]:
for p in model.parameters():
  p.requires_grad = False  

# Parameters of newly created `Linear` layer have `requires_grad=true`

In [ ]:
model.classifier[1] = pt.nn.Linear(num_ftrs, len(class_names))
model.classifier[1].weight

# Implement a generic `train` function with a validation phase

In [ ]:
def train(model, loss_fn, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with pt.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = pt.max(outputs, 1)
                    loss = loss_fn(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += pt.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase}: Epoch {epoch}/{num_epochs - 1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Re-train the last layer while monitoring for accuracy


In [ ]:
model = model.to(device)
optimizer = pt.optim.SGD(model.classifier[1].parameters(), lr=0.001, momentum=0.9)
scheduler = pt.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train(model, 
              pt.nn.CrossEntropyLoss(),
              #only parameters of final layer are being optimized 
              optimizer,
              scheduler,
              num_epochs=25)

train: Epoch 5/24 Loss: 0.5174 Acc: 0.8080
